<a href="https://colab.research.google.com/github/danicrg/whatsapp-analyser/blob/master/whatsapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whatsapp Chat Analyser

In [ ]:
!pip install emoji

In [ ]:
import re
import regex
import pandas as pd
import numpy as np
import emoji

## Parsing The Chat

In [ ]:
f = open('chat.txt', 'r')

In [ ]:
line = f.readline().strip()

In [ ]:
def startsWithDateAndTimeAndroid(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

def startsWithDateAndTimeios(s):
    pattern = '^\[([0-9]+)([\/-])([0-9]+)([\/-])([0-9]+)[,]? ([0-9]+):([0-9][0-9]):([0-9][0-9])?[ ]?(AM|PM|am|pm)?\]' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [ ]:
def findAuthor(s):
  s=s.split(":")
  if len(s)==2:
    return True
  else:
    return False

In [ ]:
def getDataPointAndroid(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    if findAuthor(message): 
        splitMessage = message.split(':') 
        author = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    return date, time, author, message

def getDataPointios(line):
	splitLine = line.split('] ')
	dateTime = splitLine[0]
	if ',' in dateTime:
		date, time = dateTime.split(',')
	else:
		date, time = dateTime.split(' ')
	message = ' '.join(splitLine[1:])
	if findAuthor(message):
		splitMessage = message.split(':')
		author = splitMessage[0]
		message = ' '.join(splitMessage[1:])
	else:
		author = None
	if time[5]==":":
		time = time[:5]+time[-3:]
	else:
		if 'AM' in time or 'PM' in time:
			time = time[:6]+time[-3:]
		else:
			time = time[:6]
	return date, time, author, message

In [ ]:
def dateconv(date):
  year=''
  if '-' in date:
    year = date.split('-')[2]
    if len(year) == 4:
      return datetime.datetime.strptime(date, "[%d-%m-%Y").strftime("%Y-%m-%d")
    elif len(year) ==2:
      return datetime.datetime.strptime(date, "[%d-%m-%y").strftime("%Y-%m-%d")
  elif '/' in date:
    year = date.split('/')[2]
    if len(year) == 4:
      return datetime.datetime.strptime(date, "[%d/%m/%Y").strftime("%Y-%m-%d")
    if len(year) ==2:
      return datetime.datetime.strptime(date, "[%d/%m/%y").strftime("%Y-%m-%d")

In [ ]:
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

In [ ]:
parsedData = []
conversationPath = 'chat.txt' 
with open(conversationPath, encoding="utf-8") as fp:
    device=''
    first=fp.readline()
    if '[' in first:
      device='ios'
    else:
      device="android"
    fp.readline() 
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        if device=="ios":
          line = line.strip()
          if startsWithDateAndTimeios(line):
            if len(messageBuffer) > 0:
              parsedData.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDataPointios(line)
            messageBuffer.append(message)
          else:
            line= (line.encode('ascii', 'ignore')).decode("utf-8")
            if startsWithDateAndTimeios(line):
              if len(messageBuffer) > 0:
                parsedData.append([date, time, author, ' '.join(messageBuffer)])
              messageBuffer.clear()
              date, time, author, message = getDataPointios(line)
              messageBuffer.append(message)
            else:
              messageBuffer.append(line)
        else:
          line = line.strip()
          if startsWithDateAndTimeAndroid(line):
            if len(messageBuffer) > 0:
              parsedData.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDataPointAndroid(line)
            messageBuffer.append(message)
          else:
            messageBuffer.append(line)

In [ ]:
if device =='android':
        df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])
        df["Date"] = pd.to_datetime(df["Date"])
        df = df.dropna()
        df["emoji"] = df["Message"].apply(split_count)
        URLPATTERN = r'(https?://\S+)'
        df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
else:
        df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])
        df = df.dropna()
        df["Date"] = df["Date"].apply(dateconv)
        df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
        df["emoji"] = df["Message"].apply(split_count)
        URLPATTERN = r'(https?://\S+)'
        df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()

## General Chat Stats

In [ ]:
total_messages = df.shape[0]

dfnm = df[~df['Message'].str.contains('<Media omitted>')].copy()

media_messages = total_messages - dfnm.shape[0]

emojis = sum(df['emoji'].str.len())

links = np.sum(df.urlcount)

In [ ]:
print("Group Wise Stats")
print("Messages:",total_messages)
print("Media:",media_messages)
print("Emojis:",emojis)
print("Links:",links)

## Stats per Author

In [ ]:
authors = df.Author.unique()
print('Authors of the chat:\n')
for author in authors:
  print(author)

In [ ]:
results = pd.DataFrame(index=authors)

In [ ]:
results['numberOfMessages'] = dfnm.groupby('Author')['Message'].count()

print('Number of messages per author:\n')
print(results['numberOfMessages'])

In [ ]:
print('Media messages:\n')

results['numberOfMediaMessages'] = df[df['Message'].str.contains('<Media omitted>')].groupby('Author')['Message'].count()
results['numberOfMediaMessages']

In [ ]:
dfnm['MessageLength'] = dfnm['Message'].str.split().str.len()
results['words'] = dfnm.groupby('Author')['MessageLength'].sum()

In [ ]:
results['wordsPerMessage'] = results['words'] / results['numberOfMessages']
print('Average words per message:\n')
print(results['wordsPerMessage'])

In [ ]:
print('Number of emojis:\n')

dfnm['nEmojis'] = dfnm['emoji'].apply(len)
results['nEmojis'] = dfnm.groupby('Author')['nEmojis'].sum()
results['nEmojis']

In [ ]:
results['wordsPerMessageNoEmojis'] = (results['words'] - results['nEmojis']) / results['numberOfMessages']

In [ ]:
def countFrequency(my_list): 
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

In [ ]:
def sortDict(dictionary):
    sorted_dictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_dictionary)

In [ ]:
results['emoji'] = dfnm.groupby('Author')['emoji'].sum().apply(countFrequency).apply(sortDict)

In [ ]:
results

In [ ]:
import matplotlib.pyplot as plt

row = results.emoji[0]

plt.bar(list(row)[:5], list(row.values())[:5])